# Base Load

In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import dask.dataframe as dd
from tqdm import trange
import pickle

##  data

In [12]:
df = dd.read_parquet("./訓練資料集/Cut_for_13dt_24dt").compute()

In [13]:
df.head(5)

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,13.0,10115966,15,1.0,9807.041135,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
1,13.0,10115966,18,3.0,19370.227515,1.0,2.0,0.0,0.0,0.23,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
2,13.0,10115966,21,1.0,9754.222977,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
3,13.0,10115966,37,8.0,17144.945999,1.0,7.0,0.0,0.0,0.12,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
4,13.0,10294163,36,1.0,5773.036485,1.0,0.0,0.0,0.0,1.00,...,2.0,2.0,11.0,1.0,2.0,30.0,68561.179121,0.0,3.0,1.0


# KMeans 分群 -- 50群

In [42]:
df2 = df[['chid', 'txn_amt', 'txn_cnt']]
df2 = df2.groupby(by=['chid']).sum()
df2 = df2.reset_index()

In [49]:
no_data = list(set(range(10000000, 10500000)) - set(df2.chid))
print(len(no_data))

for id_ in no_data:
    df2 = df2.append(pd.DataFrame({'chid':[id_], 'txn_amt':[0], 'txn_cnt':[0]}))
    
df2 = df2.sort_values(by=['chid'])

4402


In [50]:
df2.shape

(500000, 3)

In [54]:
from sklearn.cluster import KMeans
cluster = 50
kmeans = KMeans(n_clusters=cluster, random_state=0).fit(df2)
label = kmeans.labels_

In [55]:
df2['label'] = label
df2.to_pickle("./訓練資料集/label_2.pkl")

In [56]:
df2.head(3)

,chid,txn_amt,txn_cnt,label
0,10000000,131047.220026,28.0,19
1,10000001,258690.163717,66.0,48
2,10000002,9264.018807,2.0,36


In [59]:
id_in_which_label = {}
for i in range(cluster):
    df3 = df2[df2['label']==i]
    id_in_which_label[i] = list(df3.chid)

In [63]:
label_dict = {}
for i in range(cluster):
    df3 = df[df['chid'].isin(id_in_which_label[i])]
    df3 = df3.groupby(by=['shop_tag']).sum().reset_index()

    a = sorted([(cnt, tag) for tag, cnt in zip(df3.shop_tag, df3.txn_cnt)], reverse=True)
    
    label_dict[i] = [tag for cnt, tag in a[:5]]

In [64]:
len(label_dict)

50

# 算上時間權重

In [72]:
df3 = df[['chid', 'shop_tag', 'dt', 'txn_amt']]
df3["dt_x_txn_amt"] = df3['dt'] * df3['txn_amt']
df3 = df3.drop(columns=['dt', 'txn_amt']).groupby(by=['chid', 'shop_tag']).sum().sort_values(by=['chid', 'dt_x_txn_amt'])

C:\Users\Rou_yi\AppData\Local\Temp/ipykernel_21344/3900630154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["dt_x_txn_amt"] = df3['dt'] * df3['txn_amt']


In [74]:
df3 = df3.reset_index()

In [75]:
df3.head()

,chid,shop_tag,dt_x_txn_amt
0,10000000,48,2.197461e+05
1,10000000,12,5.593065e+05
2,10000000,10,1.539215e+06
3,10000001,22,8.680454e+04
4,10000001,36,1.228687e+05


# 抓取 tag

In [80]:
chid_ls = np.array(df3.chid)
shop_tag_ls = np.array(df3.shop_tag)
txn_amt_ls = np.array(df3.dt_x_txn_amt)

In [88]:
all_ls = []
for i in trange(500000):
    id_ = (10000000+i)
    if id_ in chid_ls:
        loc = np.where(chid_ls==id_)[0]
        if len(loc) == 3:
            all_ls.append([id_] + list(shop_tag_ls[loc]))
            
        elif len(loc) < 3:
            current_tag_ls = list(shop_tag_ls[loc])
            current_tag_ls.reverse()
            tag_list = label_dict[label[i]]
            for k in range(5):
                another_tag = tag_list[k]
                if another_tag not in current_tag_ls:
                    current_tag_ls.append(another_tag)
                if len(current_tag_ls) == 3:
                    break
            all_ls.append([id_] + current_tag_ls)
 

        elif len(loc) > 3:
            current_tag_ls = list(shop_tag_ls[loc])
            current_tag_ls.reverse()
            all_ls.append([id_] + current_tag_ls[:3])
    else:
        another_tag = label_dict[label[i]][:3]
        all_ls.append([id_] + another_tag)

100%|█████████████████████████████████████████████████████████████████████████| 500000/500000 [59:34<00:00, 139.88it/s]


In [105]:
a = pd.DataFrame(all_ls, columns=['chid', 'top1', 'top2', 'top3'])
base_csv = pd.read_csv("./需預測的顧客名單及提交檔案範例.csv")[['chid']]
a = base_csv.merge(a).astype('str')

In [108]:
a.to_csv("./訓練資料集/model/1231/消費總金額計算(50群, 時間加權).csv", index=False)

【不使用模型】  

取 dt = 21-24  
加總各 ID 在各 tag 的總消費金額，並以此建立 top 3 tag  
缺少的由上頁8群的前五 tag 結果下去填補  
不在資料內的人使用 36, 37, 15 填補  
耗時 : 48:09  //  分數 : 0.6376  

取 dt = 13-24  
以消費0元填補缺少的二千多個ID，並將50萬個ID下去分50群  
加總各 ID 在各 tag 的(dt *總消費金額)，並以此建立 top 3 tag  
缺少資料的ID，從他所在的群數曲前三名 tag 結果下去填補  
耗時 : 59:34  //  分數 : 0.6640  

下方往過去時間(1-13dt)尋找消失的4402人裡，其中2442人的資料  
有消費類別與金額，再重新給予分群，從過往時間中抓取總消費前三作為top3，不足三使用分群tag填補  
分數 : 0.6640  (完全沒改變，我比對過資料是有變化的)  
使用過往資料無法改善分數。

In [ ]:
path_1 = "./訓練資料集/train_tag_filter.csv"
df = dd.read_csv(path_1, dtype={'shop_tag': 'object', 'chid': 'int'}, assume_missing=True).compute()
# 選取 ID、時間、類別、金額、次數、性別、年齡、行業、職位
df = df[['chid', 'dt', 'shop_tag', 'txn_amt', 'gender_code', 'age', 'trdtp']]
df = df[df['chid'].isin(list(chid_ls))]

In [ ]:
df = pd.DataFrame(df)
df = df.sort_values(by=['chid', 'dt', 'txn_amt']).reset_index(drop=True)

In [ ]:
df.chid.nunique()

In [ ]:
df.shape

In [ ]:
chid_ls_2 = df.chid.unique()

In [ ]:
#sixteen_tag = [2, 6, 10, 12, 13, 15, 18, 19, 21, 22, 25, 26, 36, 37, 39, 48]
collect = {}
for id_ in list(chid_ls_2):
    df2 = df[df['chid']==id_]
    a = [list(df2.gender_code)[-1], list(df2.age)[-1], list(df2.trdtp)[-1]]
    
    df2 = df2[["dt", 'shop_tag', 'txn_amt']].groupby(by=['shop_tag']).sum()
    df2 = df2.reset_index().sort_values(by=["dt", 'txn_amt'], ascending=False)
    if len(df2.shop_tag) < 3:
        collect[id_] = list(df2.shop_tag) + ['0']*(3-len(df2.shop_tag)) + [df2.txn_amt.sum()] + a 
    if len(df2.shop_tag) >= 3:
        collect[id_] = list(df2.shop_tag)[:3] + [df2.txn_amt.sum()] + a

In [ ]:
len(collect)

In [ ]:
df2 = pd.DataFrame(collect).transpose().reset_index()
df2.columns = ['chid', 'top1', 'top2', 'top3', 'txn_amt', 'gender_code', 'age', 'trdtp']
df2.head(3)

In [ ]:
#df2.to_csv("./訓練資料集/model/0103/不見的ID前三類別.csv")

In [ ]:
df = dd.read_parquet("./訓練資料集/Cut_for_13dt_24dt").compute()
df = df[['chid', 'txn_amt']]
df = df.groupby(by=['chid']).sum()
df = df.reset_index()

In [ ]:
no_data = list(set(range(10000000, 10500000)) - set(df.chid))
print(len(no_data))
id_ls = df2.chid
for id_ in no_data:
    if id_ in id_ls:
        df = df.append(pd.DataFrame({'chid':[id_], 'txn_amt':list(df2[df2.chid==id_].txn_amt)}))
    else:
        df = df.append(pd.DataFrame({'chid':[id_], 'txn_amt':[0]}))
df = df.sort_values(by=['chid'])

In [ ]:
from sklearn.cluster import KMeans
cluster = 50
kmeans = KMeans(n_clusters=cluster, random_state=0).fit(df)
label = kmeans.labels_

In [ ]:
df['label'] = label
df.to_pickle("./訓練資料集/label_3.pkl")

In [ ]:
id_in_which_label = {}
for i in range(cluster):
    df3 = df[df['label']==i]
    id_in_which_label[i] = list(df3.chid)

In [ ]:
#df2 = dd.read_parquet("./訓練資料集/Cut_for_13dt_24dt").compute()
label_dict = {}
for i in range(cluster):
    df3 = df2[df2['chid'].isin(id_in_which_label[i])]
    df3 = df3.groupby(by=['shop_tag']).sum().reset_index()

    a = sorted([(cnt, tag) for tag, cnt in zip(df3.shop_tag, df3.txn_cnt)], reverse=True)
    
    label_dict[i] = [tag for cnt, tag in a[:5]]

In [ ]:
df3 = pd.read_csv("./訓練資料集/model/0103/不見的ID前三類別.csv")
df3 = df3[['chid', 'top1', 'top2', 'top3']]
df3.head(3)

In [ ]:
df4 = pd.read_csv("./訓練資料集/model/1231/消費總金額計算(50群, 時間加權).csv")
df4.head(3)

In [ ]:
for id_ in list(df3.chid)[:2]:
    current_tag_ls = list(df3[df3.chid == id_][['top1', 'top2', 'top3']].values[0])

    if 0 in current_tag_ls:
        tag_list = label_dict[label[(id_-10000000)]].copy()
        if current_tag_ls[0] == 0:
            current_tag_ls = tag_list[:3]
            
        elif  current_tag_ls[1] == 0:
            if current_tag_ls[0] in tag_list:
                tag_list.remove(current_tag_ls[0])
            current_tag_ls[1:] = tag_list[:2]
            
        else:
            if current_tag_ls[0] in tag_list:
                tag_list.remove(current_tag_ls[0])
            if current_tag_ls[1] in tag_list:
                tag_list.remove(current_tag_ls[1])
            current_tag_ls[2] = tag_list[0]

    df4.loc[np.where(df4.chid == id_)[0], :] = [id_] + current_tag_ls

In [ ]:
df4.to_csv("./訓練資料集/model/0103/從過往時間修正一些ID的類別.csv", index=False)